# SoccerNetPlus
by AiJing Wu, Michael Vanden Heuvel

### Environment settings:
1. Create a directory/folder named "SoccerNetPlus" and put everything in it
2. Runtime -> Change runtime type -> GPU
3. Mount the drive to colab (with the next code snippet)

In [ ]:
# interact with drive files
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Important variables
max_allowed_video_length = 10
parent_dir = '/content/gdrive/My Drive/SoccerNetPlus/'

### Imports

In [ ]:
# download all dependencies
%cd $parent_dir
!pip install -r dependency-requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [ ]:
# import all libraries
from IPython.display import YouTubeVideo, display
import requests
import os, subprocess
import cv2

### Anvil Setup

In [ ]:
# link with Anvil frontend
import anvil.server
anvil.server.connect("PNLBOIT4UOOXR5MDHAOGGJI7-KSSN3T627CDJ22BL")

### User Inputs
get input video from YouTube, and put it in SoccerNetPlus/video_input
1. YouTube id (do an interface like "https://www.youtube.com/watch?v="______________)
2. start time (eg. 03:20 - minute:second)
3. end time (eg. 05:40 - minute:second)
4. video field corners' coordinates

In [ ]:
# Source: https://stackoverflow.com/questions/63325908/how-do-i-check-if-a-youtube-video-url-is-valid-or-not-in-python
def check_video_url(video_id):
    checker_url = "https://www.youtube.com/oembed?url=http://www.youtube.com/watch?v="
    video_url = checker_url + video_id

    request = requests.get(video_url)

    return request.status_code == 200
check_video_url("SB-qEYVdvXA")

In [ ]:
# get the video source and download the video
def download_youtube():
    youtube_id = "ta-M_RIHyFA"
    youtube_link = "https://www.youtube.com/watch?v=" + youtube_id
    display(YouTubeVideo(youtube_id))
    !youtube-dl $youtube_link -f 22 -o video_input/video.mp4

In [ ]:
from __future__ import unicode_literals
import youtube_dl

ydl_opts = {'opts'}
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download(['https://www.youtube.com/watch?v=BaW_jenozKc'])

### Detection and 2D Birdeye Transform
with Yolov3 & OpenCV, all running in SoccerNetPlus/yolov3_opencv

In [ ]:
def detection_transformation():
    os.chdir(os.path.join(parent_dir, "yolov3_opencv"))
    # Command: birdeye_view.py path-to-input-video path-to-output-video-detection-part path-to-output-video-birdeye-part
    subprocess.call("python birdeye_view.py ../video_input/test-video.mp4 ../video_output/detection.mp4 ../video_output/birdeye.mp4")
    os.chdir(parent_dir)

### Tracking and Statistics Display
with Yolov3 & DeepSort, all running in SoccerNetPlus/yolov3_deepsort

In [ ]:
def tracking_statistics():
    os.chdir(os.path.join(parent_dir, "yolov3_deepsort"))
    subprocess.call("python load_weights.py", shell=True)
    # Command: object_tracker.py --video path-to-input-video --output path-to-output-video
    subprocess.call("python object_tracker.py --video ../video_input/test-video.mp4 --output ../video_output/tracking.mp4", shell=True)
    os.chdir(parent_dir)

## UI Code

In [ ]:
@anvil.server.callable
def populate_example_video_dropdown(clip_duration=10):
  global video_input
  video_input = {}
  dir = os.path.join(parent_dir, "video_input")
  videos = [x for x in os.listdir(dir) if x.endswith(".mp4")]
  video_durations = []
  for video in videos:
    video = os.path.join(dir,video)
    vidcapture = cv2.VideoCapture(video)
    fps = vidcapture.get(cv2.CAP_PROP_FPS)
    totalNoFrames = vidcapture.get(cv2.CAP_PROP_FRAME_COUNT)
    video_durations.append(int(totalNoFrames / fps))

  for i, v in enumerate(zip(videos,video_durations)):
    video, duration = v
    # print(f"Duration: {duration}")
    # print(range(0,clip_duration,duration))
    # print(list(range(0,clip_duration,duration)))
    for j,s in enumerate(range(0,duration,clip_duration)):
      # print(f"Clip {((i+1)*(j+1))}, dur: {duration}")
      video_input[f"Clip {((i+1)*(j+1))}"] = {"path": os.path.join(dir, video), "start":s}
  return list(video_input.keys())
  
populate_example_video_dropdown()
# video_input

['Clip 1',
 'Clip 2',
 'Clip 3',
 'Clip 4',
 'Clip 5',
 'Clip 6',
 'Clip 7',
 'Clip 8',
 'Clip 9',
 'Clip 10',
 'Clip 11',
 'Clip 12']

In [ ]:
anvil.server.wait_forever()

KeyboardInterrupt: ignored

In [ ]:
# list(range(0,120, 10))

[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110]

In [ ]:
@anvil.server.callable
def run_soccernet_button():
  pass
  # if youtube video entered:
    # if valid yt video
      # download youtube video
      # run model on yt video
    # else
      # error message
  # elif check if example video selected from example drop down
    # run model on example video
    # name video based on timing
  # else
    # error message about no video input